# Лабораторная 2

In [9]:
from abc import ABCMeta, abstractmethod
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [10]:
df = pd.read_csv("data\creditcard.csv")

In [11]:
df.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [12]:
class Classificator():
    __metaclass__ = ABCMeta
    
    @abstractmethod
    def train(X, Y):
        '''Обучить модель'''
    
    @abstractmethod
    def predict(self, X):
        '''Предсказать данные'''

In [60]:
class DecisionTree(Classificator):
    model = None
    
    def train(self, X, Y, tabs="", depth = 0, prediction=None, max_depth=None, leafValue=None):
        entropy = self.calculate_entropy(Y)
        #print(entropy)
        #print(Y.nunique())
        node = {}
        
        if entropy == 0 or depth == max_depth or len(X) == leafValue:
            p = 0
            if len(Y.loc[Y != 1]) == 0:
                p = 1
            elif len(Y.loc[Y == 1]) == 0:
                p = 0
            else:
                p = len(Y.loc[Y == 1]) / len(Y.loc[Y != 1])
            #print(Y.value_counts())
            #self.node[prediction.attribute] = [prediction.predicate, p]
            node['label'] = p
            return node
        igs = []
        for index, row in X.iterrows():
            inner_igs = []
            for column in X:
                predicate = row[column]
                left = X[column].loc[X[column] <= predicate]
                right = X[column].loc[X[column] > predicate]
                Y_left = Y.loc[left.index]
                Y_right = Y.loc[right.index]
                groups = [[len(left), self.calculate_entropy(Y_left)], [len(right), self.calculate_entropy(Y_right)]]
                ig = self.information_gain(df.Class, entropy, groups)
                inner_igs.append(Predicate(column, ig, predicate))
            igs.append(self.get_max_ig(inner_igs))

        predicate = self.get_max_ig(igs)
        
        
        X_left = X.loc[X[predicate.attribute] <= predicate.predicate].drop(predicate.attribute, axis=1)
        X_right = X.loc[X[predicate.attribute] > predicate.predicate].drop(predicate.attribute, axis=1)
        Y_left = Y.loc[X_left.index]
        Y_right = Y.loc[X_right.index]
        depth += 1
        #print(tabs + "left")
        #left_node = DecisionTree().train(X_left, Y_left, prediction=prediction depth=depth, max_depth=max_depth)
        #print(tabs + "right")
        #right_node = DecisionTree().train(X_right, Y_right, prediction=prediction depth=depth, max_depth=max_depth)
        #left = DecisionTree().train(X_left, Y_left, prediction=predicate, depth=depth, max_depth=max_depth)
        #rigth = DecisionTree().train(X_right, Y_right, prediction=predicate, depth=depth, max_depth=max_depth)
        node['attribute'] = predicate.attribute
        node['value'] = predicate.predicate
        node['nodes'] = {}
        node['nodes']['<='] = DecisionTree().train(X_left, Y_left, depth=depth, max_depth=max_depth)
        node['nodes']['>'] = DecisionTree().train(X_right, Y_right, depth=depth, max_depth=max_depth)
        #self.node[predicate.attribute] = predicate.predicate
        return node
    
    def predict(self, X):
        pass
    
    def calculate_entropy(self, Y):
        count = len(Y)
        entropy = 0
        for count_class in Y.value_counts():
            p = count_class / count
            entropy -= p * np.log2(p)
        return entropy
    
    
    def information_gain(self, Y, entropy0, groups):
        N = len(Y)
        sum_entropies = 0
        for group in groups:
            group
            Ni = group[0]
            Si = group[1]
            sum_entropies += Ni / N * Si
        return entropy0 - sum_entropies
    
    def get_max_ig(self, igs):
        information_gain = igs[0]
        for ig in igs:
            if ig.ig > information_gain.ig:
                information_gain = ig
        return information_gain

In [14]:
class RandomForest(Classificator):
    model = None
    
    def __init__():
        pass
    
    def train(X, Y):
        pass
    
    def predict(self, X):
        pass

In [15]:
class GradientBoosting(Classificator):
    model = None
    
    def __init__():
        pass
    
    def train(X, Y):
        pass
    
    def predict(self, X):
        pass

In [65]:
X = df[df.columns[:-1]][10000:20000]
y = df.Class[10000:20000]

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tree = DecisionTree()
t = tree.train(X_train, Y_train, max_depth=3)
#print(pred.attribute, pred.ig)

In [17]:
def read_tree(t, tabs='', d=''):
    print(tabs+'predicate: ' + d)
    if 'label' in t:
        print(tabs+'label: ' + str(t['label']))
    else:
        print(tabs+'attr: ' + t['attribute'])
        print(tabs+'val: ' + str(t['value']))
        read_tree(t['nodes']['<='], tabs+'\t', d='<=')
        read_tree(t['nodes']['>'], tabs+'\t', d='>')
    #print(t['nodes']['<='])
    #print(t['nodes']['>'])
    return

In [5]:
class Predicate:
    attribute = ""
    ig = 0
    predicate = 0
    
    def __init__(self, attribute, ig, predicate):
        self.attribute = attribute
        self.ig = ig
        self.predicate = predicate

In [23]:
len(df.Class.loc[df.Class == 1])

492

In [66]:
read_tree(t)

predicate: 
attr: V9
val: -3.5657377912437798
	predicate: <=
	attr: V20
	val: 1.7068890619925001
		predicate: <=
		label: 1
		predicate: >
		attr: V1
		val: -29.876365513976303
			predicate: <=
			label: 1
			predicate: >
			label: 0
	predicate: >
	attr: V12
	val: -2.5475568142293104
		predicate: <=
		attr: V14
		val: -0.33030580375144103
			predicate: <=
			label: 0.1111111111111111
			predicate: >
			label: 0.0011325028312570782
		predicate: >
		label: 0


In [72]:
df[10000:20000].loc[df.V9 > -3.5657377912437798].loc[df.V12 <= -2.5475568142293104].loc[df.V14 > -0.33030580375144103].Class.value_counts()

0    1133
1       1
Name: Class, dtype: int64

In [28]:
df[:1000].loc[df.V9 > -2.77008927719433].loc[df.V23 > 1.30469309419231].loc[df.V12 > -0.503140859566824].Class.value_counts()

0    7
Name: Class, dtype: int64

In [24]:
df.loc[df.V9 > -2.77008927719433]

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0
